**Connecting Colab to G-Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Moving to Path Folder & Installing  Dependencies**

In [1]:
%cd /content/drive/MyDrive/Resnet18

/content/drive/MyDrive/Resnet18


In [2]:
## Importing dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os

In [ ]:
!ls

 animal_sgd_0001_model.pth		    dataset_animals.zip
 APHU6500062_20240601072327.jpg		    elephnt00005.jpg
 cat0005.jpg				    model.pth
 Container_cls_Adam_10Ep_sz1024_model.pth   Res0.ipynb
 Container_cls_Adam_60Ep_model.pth	    Res_frz4_fc_sz1024_Adm0001_CrossEnt.ipynb
 container_sgd_0001_model.pth		    Res_frz4_fc_sz256_Adm0001_CrossEnt.ipynb
 dataset				   'Screenshot (534).png'
 dataset_animals			    Test


**Data Transformations For Data Augmentation & Normalization**

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(1024),                   # Random crop to 1024x1024
        transforms.RandomHorizontalFlip(),                    # Random horizontal flip
        transforms.RandomAffine(degrees=0, shear=20, scale=(0.8, 1.2)),  # Shear and zoom (scale) augmentation
        transforms.ToTensor(),                                # Transform to pixel values [0,1]
        transforms.Normalize([0.485, 0.456, 0.406],           # Mean normalization
                             [0.229, 0.224, 0.225])           # Standard deviation normalization
    ]),
    'val': transforms.Compose([
        transforms.Resize(1024),                              # Resize the shorter side to 1024
        transforms.CenterCrop(1024),                          # Crop the center to 1024x1024
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}


**Dataset Path and Dataloader**

In [4]:
## Dataset path
data_dir = '/content/drive/MyDrive/Resnet18/dataset'

## Data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)         ## To print dataset size

class_names = image_datasets['train'].classes
print(class_names)            ## To print classes names

{'train': 353, 'val': 89}
['Clean', 'Garbage', 'Incomplete', 'Not Clean']


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


**Loading Pre-trained ResNet-18, Freezing All Layers Except layer4 & fc, Adding custom dense layers, Loss Function and Optimizer**

In [5]:
## Loading pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

## Freeze all layers except 'layer4' and 'fc'
for name, param in model.named_parameters():
    if "layer4" not in name and 'fc' not in name:
        param.requires_grad = False

## Parameters that gonna be trained
print("Parameters that gonna be trained:")
parameters_to_train = []
for name, param in model.named_parameters():
    if param.requires_grad:
        parameters_to_train.append(param)
        print("\t", name,param.requires_grad)
print("Total parameters to train:", len(parameters_to_train))

## Adding 4 FC layers
# Modify the final fully connected layer
num_ftrs = model.fc.in_features  # Number of features from the ResNet-18 backbone
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1024),    # First dense layer
    nn.ReLU(),                   # Activation for the first dense layer
    nn.Dropout(0.5),           # Dropout layer
    nn.Linear(1024, 512),        # Second dense layer
    nn.ReLU(),                   # Activation for the second dense layer
    nn.Dropout(0.5),
    nn.Linear(512, 256),         # Third dense layer
    nn.ReLU(),                   # Activation for the third dense layer
    nn.Linear(256, len(class_names)),  # Fourth dense layer
    nn.Softmax(dim=1)            # Softmax activation for output probabilities
)


## Loss function & Optimizer
criterion = nn.CrossEntropyLoss()       ## Loss function
# optimizer = optim.SGD(model.parameters(), lr=5e-7, momentum=0.9)     ## Optimizer & LR = 5x10^-7
optimizer = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

## Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 38.5MB/s]


Parameters that gonna be trained:
	 layer4.0.conv1.weight True
	 layer4.0.bn1.weight True
	 layer4.0.bn1.bias True
	 layer4.0.conv2.weight True
	 layer4.0.bn2.weight True
	 layer4.0.bn2.bias True
	 layer4.0.downsample.0.weight True
	 layer4.0.downsample.1.weight True
	 layer4.0.downsample.1.bias True
	 layer4.1.conv1.weight True
	 layer4.1.bn1.weight True
	 layer4.1.bn1.bias True
	 layer4.1.conv2.weight True
	 layer4.1.bn2.weight True
	 layer4.1.bn2.bias True
	 fc.weight True
	 fc.bias True
Total parameters to train: 17


**Model Summary**

In [6]:
!pip install torchinfo

from torchinfo import summary

## This model is our trained model
summary(model, input_size=(4, 3, 1024, 1024))  ## Batch size, No. of channels (RGB), Input size

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [4, 4]                    --
├─Conv2d: 1-1                            [4, 64, 512, 512]         (9,408)
├─BatchNorm2d: 1-2                       [4, 64, 512, 512]         (128)
├─ReLU: 1-3                              [4, 64, 512, 512]         --
├─MaxPool2d: 1-4                         [4, 64, 256, 256]         --
├─Sequential: 1-5                        [4, 64, 256, 256]         --
│    └─BasicBlock: 2-1                   [4, 64, 256, 256]         --
│    │    └─Conv2d: 3-1                  [4, 64, 256, 256]         (36,864)
│    │    └─BatchNorm2d: 3-2             [4, 64, 256, 256]         (128)
│    │    └─ReLU: 3-3                    [4, 64, 256, 256]         --
│    │    └─Conv2d: 3-4                  [4, 64, 256, 256]         (36,864)
│    │    └─BatchNorm2d: 3-5             [4, 64, 256, 256]         (128)
│    │    └─ReLU: 3-6                    [4, 64, 256, 256] 

In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

**Training The Model**

In [7]:
import matplotlib.pyplot as plt

## Dictionaries to store train and val loss and accuracy
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

num_epochs = 50                    ## Number of Epochs for training
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print('-' * 50)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        ## Store the metrics in the history dictionary
        if phase == 'train':
            history['train_loss'].append(epoch_loss)
            history['train_acc'].append(epoch_acc.item())
        else:
            history['val_loss'].append(epoch_loss)
            history['val_acc'].append(epoch_acc.item())

print("\nCongrats Training complete!")

## Plot Loss and Accuracy
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(14, 5))


## Plot Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, history['train_loss'], label='Training Loss')
plt.plot(epochs, history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

## Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history['train_acc'], label='Training Accuracy')
plt.plot(epochs, history['val_acc'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Epoch 1/50
--------------------------------------------------


KeyboardInterrupt: 

**Saving The Model**

In [ ]:
## Saving the model
# torch.save(model.state_dict(), 'Container_classifier_SGD_model.pth')
torch.save(model.state_dict(), 'Container_cls_Adam_10Ep_sz1024_model.pth')

In [ ]:
## Loading the model
loaded_model = SimpleCNN()
loaded_model.load_state_dict(torch.load('Container_cls_Adam_10Ep_sz1024_model.pth'))
print(loaded_model)

NameError: name 'SimpleCNN' is not defined

In [ ]:
# checkpoint = torch.load('Container_cls_Adam_60Ep_model.pth')
# print(checkpoint.keys())  ## Prints all the keys in the state_dictcheckpoint

<ipython-input-20-c4d0a7de7176>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('Container_cls_Adam_60Ep_model.pth')


odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.num_batches_tracked', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', '

In [ ]:
torch.save(model, 'model.pth')

In [ ]:
# import torch
# from torchvision import models, transforms
# from PIL import Image

# # Load the saved model
# model = models.resnet18(pretrained=True)
# model.fc = nn.Linear(model.fc.in_features, 1000)  # Adjust to match the original model's output units
# model.load_state_dict(torch.load('Container_cls_Adam_10Ep_sz1024_model.pth'))
# model.eval()

# # Create a new model with the correct final layer
# new_model = models.resnet18(pretrained=True)
# new_model.fc = nn.Linear(new_model.fc.in_features, 4)  # Adjust to match the desired output units

# Copy the weights and biases from the loaded model to the new model
# new_model.layer4.0.conv1.weight.data = model.layer4.0.conv1.weight.data[0:2]
# new_model.layer4.0.bn1.weight.data = model.layer4.0.bn1.weight.data[0:2]
# new_model.layer4.0.bn1.bias.data = model.layer4.0.bn1.bias.data[0:2]
# new_model.layer4.0.conv2.weight.data = model.layer4.0.conv2.weight.data[0:2]
# new_model.layer4.0.bn2.weight.data = model.layer4.0.bn2.weight.data[0:2]

# new_model.layer4.0.bn2.bias.data = model.layer4.0.bn2.bias.data[0:2]
# new_model.layer4.0.downsample.0.weight.data = model.layer4.0.downsample.0.weight.data[0:2]


# new_model.layer4.0.downsample.1.weight.data = model.layer4.0.downsample.1.weight.data[0:2]
# new_model.layer4.0.downsample.1.bias.data = model.layer4.0.downsample.1.bias.data[0:2]
# new_model.layer4.1.conv1.weight.data = model.layer4.1.conv1.weight.data[0:2]
# new_model,layer4.1.bn1.weight.data = model.layer4.1.bn1.weight.data[0:2]
# new_model.layer4.1.bn1.bias.data = model.layer4.1.bn1.bias.data[0:2]
# new_model.layer4.1.conv2.weight.data = model.layer4.1.conv2.weight.data[0:2]
# new_model.layer4.1.bn2.weight.data = model.layer4.1.bn2.weight.data[0:2]
# new_model.layer4.1.bn2.bias.data = model.layer4.1.bn2.bias.data[0:2]

# new_model.fc.weight.data = model.fc.weight.data[0:2]  # Copy only the first 2 output units
# new_model.fc.bias.data = model.fc.bias.data[0:2]

<ipython-input-14-952799b4441c>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Container_cls_Adam_10Ep_sz1024_model.pth'))


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "fc.0.weight", "fc.0.bias", "fc.3.weight", "fc.3.bias", "fc.6.weight", "fc.6.bias", "fc.8.weight", "fc.8.bias". 

**Model Summary**

In [ ]:
# import torch
# from torchvision import transforms
# from PIL import Image

# import torch
# import torch.nn as nn
# from torchvision import models
# from torchvision.transforms import Compose, Resize, ToTensor, Normalize
# from PIL import Image

# # Step 1: Define the model architecture
# class ContainerClassifier(nn.Module):
#     def __init__(self, num_classes=4):
#         super(ContainerClassifier, self).__init__()
#         self.backbone = models.resnet18(pretrained=False)
#         self.backbone.fc = nn.Sequential(
#             nn.Linear(512, 256),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(128, num_classes),
#             nn.Softmax(dim=1)
#         )

#     def forward(self, x):
#         return self.backbone(x)

# # Step 2: Create an instance of the model and load weights
# model = ContainerClassifier(num_classes=4)
# model.load_state_dict(torch.load('Container_cls_Adam_60Ep_model.pth'), strict=False)
# model.eval()

# # Step 3: Define preprocessing for input images
# transform = Compose([
#     Resize((256, 256)),  # Resize to match input size
#     ToTensor(),
#     Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
# ])

# # Step 4: Load and preprocess an image
# image_path = '/content/drive/MyDrive/Resnet18/Test/img0005.jpg'  # Path to your test image
# image = Image.open(image_path).convert('RGB')  # Open and convert to RGB
# input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# # Step 5: Perform inference
# with torch.no_grad():
#     output = model(input_tensor)
#     predicted_class = torch.argmax(output, dim=1).item()

# print(f"Predicted Class: {predicted_class}")



# ['Clean', 'Garbage', 'Incomplete', 'Not Clean']
# Map class index to label (if applicable)
# class_labels = {0: "Clean", 1: "Garbage", 2: "Incomplete", 3: "Not Clean"}
# print(f"Predicted class: {class_labels[predicted_class.item()]}")


Predicted Class: 3


<ipython-input-22-c953d83df7cd>:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Container_cls_Adam_60Ep_model.pth'), strict=False)


<!--  -->